In [2]:
import os
import sys
from tqdm import tqdm

src_path = os.path.abspath(os.path.join('../../', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from tools.data_processor import DataProcessor
from tools.frame_processors import SupervisionVertexProcessorWithLandmarkFrontalization
from tools.frame_preprocessors import TextureFrontalizationPreprocessor, FaceExtractionPreprocessor

In [3]:
DATA_PATH = os.path.abspath(os.path.join('..', '..', 'data', 'raw', 'real_life_deception_detection', 'Real-life_Deception_Detection_2016', 'Clips'))
PROCESSED_DATA_PATH = os.path.abspath(os.path.join('..', '..', 'data', 'processed', 'real_life_deception_detection'))
REFERENCE_POINTS_PATH = os.path.abspath(os.path.join('..', '..', 'data', 'reference_points', 'key_points_xyz.npy'))

MODELS_PATH = os.path.join(src_path, 'models', 'frontalization_models')


### Data processor

In [4]:
dp = DataProcessor(
    frame_preprocessors=[
        FaceExtractionPreprocessor(
            skip_bad_frames=False,
            output_size=(200, 200)
        ),
        TextureFrontalizationPreprocessor(
            models_path=MODELS_PATH,
            do_calculate_symmetry=True
        ),
    ],
    frame_processor=SupervisionVertexProcessorWithLandmarkFrontalization(
        reference_points_path=REFERENCE_POINTS_PATH,
        do_make_face_mesh=False  # Output pure landmarks
    ),
)

# Process miami data

In [5]:
# os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

In [6]:
already_processed_data = set(file.split(".")[0] for file in os.listdir(PROCESSED_DATA_PATH))

In [9]:
import os

SUBFOLDERS = ['Deceptive', 'Truthful']
ALL_CLIPS = []

for subfolder in SUBFOLDERS:
    folder_path = os.path.join(DATA_PATH, subfolder)
    clips = [(subfolder, clip) for clip in os.listdir(folder_path)]
    ALL_CLIPS.extend(clips)

In [11]:
total = len(ALL_CLIPS)

for file in tqdm(ALL_CLIPS, desc=f"Processing files", total=total, leave=False):
    file_path = os.path.join(DATA_PATH, *file)
    out_file = file[1].split(".")[0]
    if out_file in already_processed_data:
        continue
    try:
        dp.process_data(file_path, os.path.join(PROCESSED_DATA_PATH, out_file))
    except Exception as e:
        print(f"Could not process {file_path} due to {e}")
        continue